In [1]:
import h5py
import pandas as pd
import numpy as np
import os
import time

def explore_hdf5_structure(filename):
    """Explore the structure of the HDF5 file"""
    print(f"Exploring structure of {filename}...\n")
    
    with h5py.File(filename, 'r') as f:
        print("Top-level keys:", list(f.keys())[:5], "... (showing first 5)")
        print(f"Total groups: {len(f.keys())}")
        print()
        
        # Check first planet group
        if len(f.keys()) > 0:
            sample_key = list(f.keys())[0]
            print(f"Sample planet: {sample_key}")
            print(f"Available keys in planet group: {list(f[sample_key].keys())}")
            
            # Show instrument_spectrum details
            if 'instrument_spectrum' in f[sample_key]:
                spec = f[sample_key]['instrument_spectrum']
                print(f"\ninstrument_spectrum details:")
                print(f"  Shape: {spec.shape}")
                print(f"  Dtype: {spec.dtype}")
                print(f"  First 5 values: {np.array(spec)[:5]}")
                print(f"  Last 5 values: {np.array(spec)[-5:]}")
            print()

def safe_save_csv(df, output_csv, max_retries=3):
    """
    Safely save CSV with retry mechanism and timestamp-based naming if file is locked
    """
    for attempt in range(max_retries):
        try:
            df.to_csv(output_csv, index=False)
            print(f"\n✓ Successfully saved: {output_csv}")
            return output_csv
        except PermissionError:
            if attempt < max_retries - 1:
                print(f"\n⚠ File is open in another program. Retrying in 2 seconds... (Attempt {attempt + 1}/{max_retries})")
                time.sleep(2)
            else:
                # Create alternative filename with timestamp
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                base_name = output_csv.rsplit('.', 1)[0]
                extension = output_csv.rsplit('.', 1)[1] if '.' in output_csv else 'csv'
                alternative_name = f"{base_name}_{timestamp}.{extension}"
                
                print(f"\n⚠ Original file '{output_csv}' is locked (open in another program)")
                print(f"✓ Saving as: {alternative_name}")
                
                df.to_csv(alternative_name, index=False)
                return alternative_name
    
    return None

def verify_planet_id_matching(spectra_df, targets_df):
    """
    Verify that planet_IDs match correctly between spectra and targets
    """
    print(f"\n" + "=" * 70)
    print("Verifying Planet ID Matching:")
    print("=" * 70)
    
    spectra_ids = set(spectra_df['planet_ID'].values)
    targets_ids = set(targets_df['planet_ID'].values)
    
    print(f"Spectra DataFrame has {len(spectra_ids)} unique planet_IDs")
    print(f"Targets DataFrame has {len(targets_ids)} unique planet_IDs")
    
    # Check for IDs in spectra but not in targets
    missing_in_targets = spectra_ids - targets_ids
    if missing_in_targets:
        print(f"\n⚠ WARNING: {len(missing_in_targets)} planet_IDs in spectra but NOT in targets!")
        print(f"  Missing IDs: {sorted(list(missing_in_targets))[:10]}{'...' if len(missing_in_targets) > 10 else ''}")
    else:
        print(f"✓ All spectra planet_IDs found in targets")
    
    # Check for IDs in targets but not in spectra
    missing_in_spectra = targets_ids - spectra_ids
    if missing_in_spectra:
        print(f"\n⚠ WARNING: {len(missing_in_spectra)} planet_IDs in targets but NOT in spectra!")
        print(f"  Missing IDs: {sorted(list(missing_in_spectra))[:10]}{'...' if len(missing_in_spectra) > 10 else ''}")
    else:
        print(f"✓ All target planet_IDs found in spectra")
    
    # Check order matching
    common_ids = sorted(spectra_ids & targets_ids)
    if len(common_ids) > 0:
        print(f"\n✓ {len(common_ids)} planet_IDs match between both datasets")
        
        # Verify first few matches
        print(f"\nVerifying first 5 planet_IDs match:")
        for i in range(min(5, len(common_ids))):
            pid = common_ids[i]
            spec_idx = spectra_df[spectra_df['planet_ID'] == pid].index[0]
            targ_idx = targets_df[targets_df['planet_ID'] == pid].index[0]
            print(f"  planet_ID {pid}: spectra row {spec_idx}, targets row {targ_idx} {'✓' if spec_idx == targ_idx else '⚠ MISMATCH'}")
    
    print("=" * 70)
    return len(missing_in_targets) == 0 and len(missing_in_spectra) == 0

def extract_all_spectra_with_targets(hdf5_filename, 
                                     targets_csv='FM_Parameter_Table.csv',
                                     output_csv='complete_dataset.csv'):
    """
    Extract all planets' spectra from HDF5 file in ORDER and combine with targets
    IMPORTANT: Spectral values are REVERSED (spec_0 contains original position 51, spec_51 contains original position 0)
    Output format: planet_ID, spec_0, ..., spec_51, planet_temp, log_H2O, log_CO2, log_CH4, log_CO, log_NH3
    """
    
    try:
        # First explore structure
        explore_hdf5_structure(hdf5_filename)
        
        # Load target values
        print(f"Loading target values from {targets_csv}...")
        targets_df = pd.read_csv(targets_csv)
        print(f"Loaded {len(targets_df)} target rows")
        print(f"Target columns: {list(targets_df.columns)}")
        
        # Verify planet_ID column exists in targets
        if 'planet_ID' not in targets_df.columns:
            print(f"\n⚠ ERROR: 'planet_ID' column not found in {targets_csv}")
            print(f"Available columns: {list(targets_df.columns)}")
            return None
        
        print(f"Target planet_ID range: {targets_df['planet_ID'].min()} to {targets_df['planet_ID'].max()}")
        print()
        
        with h5py.File(hdf5_filename, 'r') as f:
            # Get keys in ORIGINAL ORDER (no sorting to preserve order)
            all_planet_keys = list(f.keys())
            total_planets = len(all_planet_keys)
            
            print(f"Total planets in HDF5 file: {total_planets}")
            print(f"Total planets in targets CSV: {len(targets_df)}")
            print(f"Extracting all spectra in ORIGINAL ORDER...\n")
            
            # Initialize list to collect spectra
            all_spectra = []
            planet_indices = []
            
            successful = 0
            failed = 0
            
            # Extract spectra from each planet IN ORDER
            for idx, planet_key in enumerate(all_planet_keys):
                if (idx + 1) % 5000 == 0:
                    print(f"  Processed {idx + 1}/{total_planets} planets...")
                
                try:
                    planet_group = f[planet_key]
                    
                    # Extract spectrum using 'instrument_spectrum' key
                    if 'instrument_spectrum' in planet_group:
                        # Get raw spectrum values without any modification
                        spectrum = np.array(planet_group['instrument_spectrum'][:])
                        
                        # REVERSE the spectrum array
                        # Now spec[0] will contain what was at position 51
                        # And spec[51] will contain what was at position 0
                        spectrum = spectrum[::-1]
                        
                    else:
                        raise ValueError(f"'instrument_spectrum' not found in {planet_key}")
                    
                    # Ensure correct shape (52 wavelengths)
                    if len(spectrum) == 52:
                        all_spectra.append(spectrum)
                        planet_indices.append(idx)  # Use index as planet_ID (0, 1, 2, ...)
                        successful += 1
                    else:
                        print(f"Warning: Incorrect spectrum length for {planet_key}: {len(spectrum)} (expected 52)")
                        failed += 1
                
                except Exception as e:
                    print(f"Error processing {planet_key}: {e}")
                    failed += 1
                    continue
            
            # Convert to numpy array
            spectra_array = np.array(all_spectra, dtype=np.float64)  # Shape: (n_planets, 52)
            
            print(f"\n" + "=" * 70)
            print("Extraction Complete!")
            print("=" * 70)
            print(f"Successfully extracted: {successful} planets")
            print(f"Failed: {failed} planets")
            print(f"\nFinal spectra array shape: {spectra_array.shape}")
            print(f"Spectra dtype: {spectra_array.dtype}")
            print(f"Spectra planet_ID range: {min(planet_indices)} to {max(planet_indices)}")
            print(f"\n⚠ NOTE: Spectral values have been REVERSED!")
            print(f"   spec_0 contains original position 51")
            print(f"   spec_51 contains original position 0")
            
            # Create DataFrame with spectral columns named spec_0 to spec_51
            spectral_cols = [f'spec_{i}' for i in range(52)]
            spectra_df = pd.DataFrame(spectra_array, columns=spectral_cols)
            
            # Add planet_ID as first column (simple numeric index)
            spectra_df.insert(0, 'planet_ID', planet_indices)
            
            # Show example of reversal
            print(f"\n" + "=" * 70)
            print("Example of Spectral Reversal (First Planet):")
            print("=" * 70)
            print(f"First 3 values in spec_0, spec_1, spec_2: {spectra_df.iloc[0, 1:4].values}")
            print(f"Last 3 values in spec_49, spec_50, spec_51: {spectra_df.iloc[0, -3:].values}")
            print(f"(spec_0 now contains what was originally at the end)")
            
            # VERIFY MATCHING before merging
            matching_ok = verify_planet_id_matching(spectra_df, targets_df)
            
            if not matching_ok:
                print("\n⚠ WARNING: Planet ID mismatch detected!")
                print("Proceeding with merge, but some planets may have missing targets...")
            
            # Merge with targets based on planet_ID
            print(f"\nMerging spectra with target values on 'planet_ID'...")
            combined_df = pd.merge(spectra_df, targets_df, on='planet_ID', how='left')
            
            # Check for NaN values after merge (indicates missing matches)
            target_cols = ['planet_temp', 'log_H2O', 'log_CO2', 'log_CH4', 'log_CO', 'log_NH3']
            nan_counts = combined_df[target_cols].isnull().sum()
            
            if nan_counts.sum() > 0:
                print(f"\n⚠ WARNING: Found NaN values after merge!")
                print("NaN counts by column:")
                for col in target_cols:
                    if nan_counts[col] > 0:
                        print(f"  {col}: {nan_counts[col]} missing values")
                print("\nThis means some planet_IDs in spectra don't have matching targets!")
            else:
                print(f"✓ All planets successfully matched with targets (no NaN values)")
            
            # Reorder columns: planet_ID, spec_0 to spec_51, then targets
            column_order = ['planet_ID'] + spectral_cols + target_cols
            combined_df = combined_df[column_order]
            
            # Show sample of merged data
            print(f"\n" + "=" * 70)
            print("Sample of Merged Data (first 3 rows):")
            print("=" * 70)
            print(combined_df[['planet_ID', 'spec_0', 'spec_1', 'spec_51', 'planet_temp', 'log_H2O']].head(3))
            
            # Save as CSV with safe save mechanism
            print(f"\nSaving complete dataset...")
            saved_filename = safe_save_csv(combined_df, output_csv)
            
            if saved_filename:
                print(f"\n" + "=" * 70)
                print("Save Successful!")
                print("=" * 70)
                print(f"  File: {saved_filename}")
                print(f"  Shape: {combined_df.shape}")
                print(f"  Column order: planet_ID + 52 spectra (REVERSED) + 6 targets = {combined_df.shape[1]} total")
                
                # Calculate file size
                file_size = os.path.getsize(saved_filename)
                print(f"  File size: {file_size / (1024**2):.2f} MB")
            
            # Print column order
            print(f"\n" + "=" * 70)
            print("Column Order:")
            print("=" * 70)
            print(f"1. planet_ID")
            print(f"2-53. spec_0 to spec_51 (52 spectral columns - VALUES REVERSED)")
            print(f"54-59. {', '.join(target_cols)}")
            print(f"\nFirst 5 columns: {list(combined_df.columns[:5])}")
            print(f"Last 6 columns: {list(combined_df.columns[-6:])}")
            
            # Print summary statistics for spectra
            print(f"\n" + "=" * 70)
            print("Spectral Data Statistics (first 5 wavelengths):")
            print("=" * 70)
            print(combined_df[spectral_cols[:5]].describe())
            
            # Print summary statistics for targets
            print(f"\n" + "=" * 70)
            print("Target Variables Statistics:")
            print("=" * 70)
            print(combined_df[target_cols].describe())
            
            return combined_df
    
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return None

def load_complete_dataset(csv_filename='complete_dataset.csv'):
    """
    Load the complete dataset with spectra and targets
    NOTE: Spectral values are reversed (spec_0 contains original position 51)
    """
    df = pd.read_csv(csv_filename)
    print(f"Loaded complete dataset: {df.shape}")
    print(f"Columns: planet_ID + 52 spectra (REVERSED) + 6 targets = {df.shape[1]}")
    
    # Separate into components
    planet_ids = df['planet_ID'].values
    spectra_cols = [f'spec_{i}' for i in range(52)]
    target_cols = ['planet_temp', 'log_H2O', 'log_CO2', 'log_CH4', 'log_CO', 'log_NH3']
    
    spectra = df[spectra_cols].values
    targets = df[target_cols].values
    
    print(f"\nSeparated data:")
    print(f"  Planet IDs: {planet_ids.shape}")
    print(f"  Spectra: {spectra.shape}")
    print(f"  Targets: {targets.shape}")
    print(f"\n⚠ Remember: Spectral values are REVERSED!")
    
    return df, planet_ids, spectra, targets

# Main execution
if __name__ == "__main__":
    hdf5_file = 'SpectralData.hdf5'
    targets_file = 'FM_Parameter_Table.csv'
    output_csv_file = 'complete_dataset.csv'
    
    print("=" * 70)
    print("ABC Dataset: Complete Extraction with Targets")
    print("=" * 70)
    print("Output Format:")
    print("  planet_ID, spec_0, spec_1, ..., spec_51, planet_temp, log_H2O, log_CO2, log_CH4, log_CO, log_NH3")
    print("\nFeatures:")
    print("  ✓ Maintains original order (no randomization)")
    print("  ✓ Uses numeric planet_ID (0, 1, 2, ...)")
    print("  ✓ Spectral columns first: spec_0 to spec_51")
    print("  ⚠ SPECTRAL VALUES REVERSED: spec_0 has original position 51, spec_51 has original position 0")
    print("  ✓ Target columns last: 6 targets")
    print("  ✓ Preserves original spectral values (but reversed)")
    print("  ✓ VERIFIES planet_ID matching before merge")
    print("=" * 70 + "\n")
    
    print("⚠ IMPORTANT: Close 'complete_dataset.csv' if it's open in Excel or any other program!")
    print("=" * 70 + "\n")
    
    # Extract and combine all data
    complete_df = extract_all_spectra_with_targets(
        hdf5_file, 
        targets_csv=targets_file,
        output_csv=output_csv_file
    )
    
    if complete_df is not None:
        print("\n" + "=" * 70)
        print("✓ Extraction successful!")
        print("=" * 70)

ABC Dataset: Complete Extraction with Targets
Output Format:
  planet_ID, spec_0, spec_1, ..., spec_51, planet_temp, log_H2O, log_CO2, log_CH4, log_CO, log_NH3

Features:
  ✓ Maintains original order (no randomization)
  ✓ Uses numeric planet_ID (0, 1, 2, ...)
  ✓ Spectral columns first: spec_0 to spec_51
  ⚠ SPECTRAL VALUES REVERSED: spec_0 has original position 51, spec_51 has original position 0
  ✓ Target columns last: 6 targets
  ✓ Preserves original spectral values (but reversed)
  ✓ VERIFIES planet_ID matching before merge

⚠ IMPORTANT: Close 'complete_dataset.csv' if it's open in Excel or any other program!

Exploring structure of SpectralData.hdf5...

Top-level keys: ['Planet_0', 'Planet_1', 'Planet_10', 'Planet_100', 'Planet_1000'] ... (showing first 5)
Total groups: 91392

Sample planet: Planet_0
Available keys in planet group: ['instrument_noise', 'instrument_spectrum', 'instrument_width', 'instrument_wlgrid']

instrument_spectrum details:
  Shape: (52,)
  Dtype: float64
  

In [2]:
df = pd.read_csv('complete_dataset.csv')
df

,planet_ID,spec_0,spec_1,spec_2,spec_3,spec_4,spec_5,spec_6,spec_7,spec_8,...,spec_48,spec_49,spec_50,spec_51,planet_temp,log_H2O,log_CO2,log_CH4,log_CO,log_NH3
0,0,0.001040,0.000994,0.001089,0.001204,0.001129,0.001251,0.001177,0.001334,0.001226,...,0.001434,0.001462,0.001456,0.001671,1108.725067,-6.484809,-7.016512,-3.294729,-3.351913,-6.955828
1,1,0.000699,0.000677,0.000662,0.000678,0.000670,0.000676,0.000686,0.000699,0.000708,...,0.000732,0.000742,0.000737,0.000761,636.858130,-8.829193,-8.603965,-5.501124,-4.799371,-6.020019
2,2,0.010417,0.010272,0.010255,0.010478,0.010280,0.010598,0.010380,0.010762,0.010592,...,0.010930,0.010988,0.010980,0.011338,420.527609,-6.626830,-6.420932,-6.755727,-5.318471,-4.695118
3,3,0.014896,0.014851,0.014816,0.014830,0.014819,0.014877,0.014854,0.014878,0.014920,...,0.014953,0.014957,0.014988,0.014975,1370.906116,-7.639372,-5.877303,-4.476840,-5.200010,-6.240726
4,4,0.000981,0.000938,0.000962,0.001017,0.001069,0.001099,0.001171,0.001124,0.001180,...,0.001351,0.001407,0.001393,0.001459,746.170076,-4.172605,-6.899335,-6.360434,-4.140252,-8.896858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91387,91387,0.000457,0.000416,0.000418,0.000480,0.000501,0.000548,0.000565,0.000609,0.000567,...,0.000755,0.000786,0.000830,0.001031,1230.274532,-5.610091,-7.173193,-5.096518,-3.397266,-4.400357
91388,91388,0.004489,0.004429,0.004561,0.004713,0.004757,0.004935,0.004941,0.004860,0.005023,...,0.005237,0.005304,0.005344,0.005365,1218.571572,-8.306286,-4.758709,-3.964018,-3.146747,-5.339986
91389,91389,0.001291,0.001190,0.001239,0.001349,0.001460,0.001543,0.001739,0.001443,0.001725,...,0.002266,0.002435,0.002314,0.001998,1125.496097,-5.410777,-8.539973,-8.684864,-5.008298,-8.062786
91390,91390,0.013766,0.013705,0.013677,0.013734,0.013721,0.013796,0.013801,0.013862,0.013854,...,0.013967,0.013982,0.013995,0.014127,818.176365,-3.280500,-5.285857,-3.157887,-4.717893,-4.934644
